In [19]:
import pandas as pd
import numpy as np
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u

# Define the column names and positions for J2000 RA/Dec
names = [
    'HR', 'Name', 'RAh', 'RAm', 'RAs', 'DE-', 'DEd', 'DEm', 'DEs', 'Vmag'
]
col_starts = [0, 10, 75, 77, 79, 83, 84, 86, 88, 102]
col_ends   = [4, 24, 76, 78, 82, 83, 85, 87, 89, 106]

# Read only the J2000 RA/Dec columns
catalog = Table.read(
    '../data/raw/catalog.gz',
    format='ascii.fixed_width_no_header',
    names=names,
    col_starts=col_starts,
    col_ends=col_ends,
    fill_values=[('', ' ')]
)


coords = SkyCoord(
    ra=(catalog['RAh'].astype(str) + ' ' + 
        catalog['RAm'].astype(str) + ' ' + 
        catalog['RAs'].astype(str)), 
    dec=(catalog['DE-'].astype(str) + 
         catalog['DEd'].astype(str) + ' ' + 
         catalog['DEm'].astype(str) + ' ' + 
         catalog['DEs'].astype(str)),
    unit=(u.hourangle, u.deg)
)

# Add a single column with the combined coordinate as a string
catalog['RA_DEC'] = coords.to_string('hmsdms')

catalog.pprint(max_lines=20)


 HR       Name     RAh RAm RAs  DE- DEd DEm DEs Vmag         RA_DEC        
---- ------------- --- --- ---- --- --- --- --- ---- ----------------------
   1    BD+44 4550  00  05 09.9   +  45  13  45 6.70 00h05m09.9s +45d13m45s
   2    BD-01 4525  00  05 03.8   -  00  30  11 6.29 00h05m03.8s -00d30m11s
   3 PscBD-06 6357  00  05 20.1   -  05  42  27 4.61 00h05m20.1s -05d42m27s
   4 PegBD+12 5063  00  05 42.0   +  13  23  46 5.51   00h05m42s +13d23m46s
   5    BD+57 2865  00  06 16.0   +  58  26  12 5.96   00h06m16s +58d26m12s
   6    CD-4914337  00  06 19.0   -  49  04  30 5.70   00h06m19s -49d04m30s
   7 CasBD+63 2107  00  06 26.5   +  64  11  46 5.59 00h06m26.5s +64d11m46s
   8    BD+28 4704  00  06 36.8   +  29  01  17 6.13 00h06m36.8s +29d01m17s
 ...           ... ... ...  ... ... ... ... ...  ...                    ...
9103 CetBD-11 6194  00  04 30.1   -  10  30  34 4.94 00h04m30.1s -10d30m34s
9104    BD+66 1679  00  04 42.0   +  67  10  00 5.67   00h04m42s +67d10m00s
9105    BD+4

In [20]:

# Create a pandas DataFrame with RA and Dec in degrees
df = pd.DataFrame({
    'hr': catalog['HR'],
    'name': catalog['Name'],
    'ra_degrees': coords.ra.deg,
    'dec_degrees': coords.dec.deg,
    'Vmag': catalog['Vmag']
})

df.to_csv('../data/processed/hr_catalog.csv', index=False)
print('Saved HR catalog to ../data/processed/hr_catalog.csv')


Saved HR catalog to ../data/processed/hr_catalog.csv


In [21]:
constellations_df = pd.read_csv('../data/raw/ConstellationLines.csv')
stars_clean = constellations_df.iloc[:, 2:].astype(str).apply(lambda x : x.str.strip()).replace('', np.nan)
constellations_clean_df = constellations_df.iloc[:, :2].join(stars_clean)
constellations_clean_df.to_csv('../data/processed/constellations.csv', index=False)
print('Saved constellations to ../data/processed/constellations.csv')

Saved constellations to ../data/processed/constellations.csv
